# Digital Images warm up demo

This is a summary of basic concepts 



In [ ]:
import numpy as np   
import matplotlib.pyplot as plt
%matplotlib inline

## Digital images

In [ ]:
from skimage import data, io

# LOAD
astro = data.astronaut() # image available in skimage
fish = io.imread('lionfish.jpg') # or you can load a custom one

#  VISUALIZE
plt.subplot(1,2,1)
plt.imshow(astro)

plt.subplot(1,2,2)
plt.imshow(fish);

In [ ]:
# scikit-image comes already with some images 
data??

Can you guess data type and shape of the images?

In [ ]:
print("image data type is: {}".format(type(astro)))
print("image shape is: {}".format(astro.shape))

As expected, an *RGB* image is stored in memory as a three dimensional array ($shape = rows \times columns \times channels$). The intensities of each color channel are saved in a separate matrix.

Do you know how to slice a <font face="TrueType">numpy.array</font>? Try to extract the three color channels from an image of choice and print their shape.

In [ ]:
R = astro[:,:,0]
G = astro[:,:,1]
B = astro[:,:,2]
print("R shape is: {}".format(R.shape))
print("G shape is: {}".format(G.shape))
print("B shape is: {}".format(B.shape))

Let's try to inspect the content of a channel, for instance printing the first 5 elements of the first 3 lines of the red one.

In [ ]:
print(R[:3,:5])

It looks like an image is stored in memory as a matrix filled with integer numbers. Can you guess the right range? Check it out in the next box.

Hint: you can use <font face="TrueType">np.max</font> and <font face="TrueType">np.min</font> on any input <font face="TrueType">numpy.array</font>.

In [ ]:
print("Max value for the red channel: {}".format(np.max(R)))
print("Min value for the red channel: {}".format(np.min(R)))

We now visualize the three channels in separate figures. In which channel do you expect the astronaut suit will have highest values?



In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
plt.imshow((R),cmap='gray');
plt.colorbar(orientation='vertical')
plt.title('Red')

plt.subplot(1,3,2)
plt.imshow((G),cmap='gray');
plt.title('Green')
plt.colorbar(orientation='vertical')

plt.subplot(1,3,3)
plt.imshow((B),cmap='gray');
plt.title('Blue')
plt.colorbar(orientation='vertical')

plt.tight_layout() # a handy command that increases spacing between subplots


## RGB to grayscale conversion

So, a color image is a collection of three matrices each one representing a different color channel. How can we represent a grayscale image? How many *color channels* do we need? We know that it's possible to encode a color image in grayscale using the following linear transformation:

$Y = 0.2125 \cdot R + 0.7154 \cdot G + 0.0721 \cdot B$

*The coefficients represent the measured intensity perception of typical trichromat humans, depending on the primaries being used; in particular, human vision is most sensitive to green and least sensitive to blue.* [cit. <a href="https://en.wikipedia.org/wiki/Grayscale">Wikipedia</a>]

The following Python function  converts the three channels of an input RGB image to float and then combines them in a grayscale encoding.

In [ ]:
def my_rgb2gray(img):
    return 0.2125 * my_uint2float(img[:,:,0]) + 0.7154 * my_uint2float(img[:,:,1]) + 0.0721 * my_uint2float(img[:,:,2])

Now we test it on one of the images above .

In [ ]:
plt.imshow(my_rgb2gray(fish), cmap='gray');

We developed our grayscale conversion utility, but as you can imagine <font face="TrueType">skimage</font> can do that for us. Import the <font face="TrueType">color</font> module from the main library and check the help function for <a href="http://scikit-image.org/docs/dev/api/skimage.color.html#skimage.color.rgb2gray"><font face="TrueType">color.rgb2gray</font></a>.

In [ ]:
from skimage import color
color.rgb2gray??

In [ ]:
plt.imshow(color.rgb2gray(fish), cmap='gray');

## Perceptive color spaces

Let's have a quick look at an alternative representation of color images, with the HSV color space

In [ ]:
from skimage.color import rgb2hsv
hsv_fish = rgb2hsv(fish)
hue_img = hsv_fish[:, :, 0]
sat_img = hsv_fish[:, :, 1]
value_img = hsv_fish[:, :, 2]


plt.figure(figsize=(20,5))

plt.subplot(1,4,1)
plt.imshow(fish);
plt.colorbar(orientation='vertical')
plt.title('Image (RGB)')

plt.subplot(1,4,2)
plt.imshow(hue_img,cmap='hsv');
plt.title('Hue')
plt.colorbar(orientation='vertical')

plt.subplot(1,4,3)
plt.imshow(sat_img,cmap='gray' );
plt.title('Saturation')
plt.colorbar(orientation='vertical')

plt.subplot(1,4,4)
plt.imshow(value_img,cmap='gray'  );
plt.title('Value')
plt.colorbar(orientation='vertical')

plt.tight_layout() # a handy command that increases spacing between subplots

## Histograms

Another strategy to check the distance between images is to take advantage of their color intensity distribution on the three channels, let's try to visualize them as histograms using the image of the lionfish.

Hint: check the documentation of  <font face="TrueType">plt.hist</font>.

In [ ]:
plt.hist??

In [ ]:
R = fish[:,:,0]
G = fish[:,:,1]
B = fish[:,:,2]

plt.figure(figsize=(8,3))

plt.subplot(1,3,1)
plt.hist(R.ravel(), density=True, color='r');
plt.ylim([0,0.03])


plt.subplot(1,3,2)
plt.hist(G.ravel(), density=True, color='g');
plt.ylim([0,0.03])

plt.subplot(1,3,3)
plt.ylim([0,0.03])
plt.hist(B.ravel(), density=True, color='b');

plt.tight_layout() # a handy command that increases spacing between subplots

From those histograms it looks like the blue channel can be used to discriminate the foreground (a lionfish) from the background. This is gonna be the goal of the next section.

## Background suppression

A <i>binary mask</i> is an simple but effective way to perform a fast background suppression. You can obtain a binary mask in several ways, let's see an example. Create $A$: a simple $3\times 3$ matrix and then print a binary mask corresponding to the positions where its values are bigger than a certain threshold.

In [ ]:
A = np.array([[2,2,2], [2,3,2], [2,4,2]])
print(A>2)

Easy, right? Now you can perform a simple background suppression identifying two thresholds from the histogram above and then representing the binary mask of the values lying in between them.

Hint: check the documentation for <font face="TrueType">np.multiply</font>.

In [ ]:
np.multiply??

In [ ]:
mask = ~np.multiply(B > 100, B < 200)
plt.imshow(mask, cmap='gray');

We may think of improving a bit our mask performing some morphological operations (**optional**).

In [ ]:
from skimage import morphology

selem = morphology.disk(8)
mask2 = morphology.dilation(mask, selem)

plt.imshow(mask2);

Now we can finally suppress the background in our image.

Hint 1: with <font face="TrueType">numpy</font> arrays you can use <font face="TrueType">bool</font> indexes.

Hint 2: to recompose an image that was previously decomposed in its three channels you can use <font face="TrueType">np.dstack</font>.

In [ ]:
np.dstack??

Hint 3: <font face="TrueType">numpy</font> has some utilities to initialize matrices and vectors, check for instance <font face="TrueType">np.ones</font> or <font face="TrueType">np.zeros</font>.

In [ ]:
np.ones??

In [ ]:
R2 = np.ones_like(R)*255
G2 = np.ones_like(G)*255
B2 = np.ones_like(B)*255

R2[mask2] = R[mask2]
G2[mask2] = G[mask2]
B2[mask2] = B[mask2]

fish2 = np.dstack((R2,G2,B2))
plt.imshow(fish2);